# Train and Test Neural Network for best Hyperparameter

This notebook is used to train and test the neural network based on one single ticker. The reason for this is to minimize the computation effort for optimizing the hyperparamneter of our neural network.

For our neural network we the open source machine learning framework Tensorflow.

<div align="center">
<img src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRCgxE_rPpXPy3QBugNifgsmfLnbGVkFKjIHfOX0IovKjv6eMU3">
</div>

**Tensorflow**

TensorFlow is an open-source machine learning library for research and production. TensorFlow offers APIs for beginners and experts to develop for desktop, mobile, web, and cloud. See the sections below to get started.

For more information please take a look under following link: [Tensorflow.com](https://www.tensorflow.org/tutorials)

# Content

 1. Import dependencies
 2. Load data
 3. Prepare our dataframe
 4. Train and test our neural network with different hyperparameters
 5. Result

<hr>

## 1. Import dependencies

In [38]:
import pandas as pd
import numpy as np
from numpy import concatenate
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import minmax_scale
from  pandas  import  DataFrame
from  pandas  import  concat
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM, Dropout, Activation
from sklearn.model_selection import train_test_split
from keras.wrappers.scikit_learn import KerasClassifier
import tensorflow as tf
from keras.backend import tensorflow_backend as K
from keras.constraints import maxnorm
from sklearn.model_selection import GridSearchCV

## 2. Load data

### Load data from build csv file for single ticker

In [39]:
df = pd.read_csv('Prepared Data/Training_set_NN1.csv')
df = df.drop('Unnamed: 0',axis=1)
df.head()

var2(t-150)  var3(t-150)  var2(t-149)  var3(t-149)  var2(t-148)  \
0     0.310566     0.075460     0.239299     0.074575     0.423775   
1     0.239299     0.074575     0.423775     0.068674     0.609032   
2     0.423775     0.068674     0.609032     0.062815     0.444209   
3     0.609032     0.062815     0.444209     0.062630     0.531393   
4     0.444209     0.062630     0.531393     0.060859     0.431934   

   var3(t-148)  var2(t-147)  var3(t-147)  var2(t-146)  var3(t-146)  \
0     0.068674     0.609032     0.062815     0.444209     0.062630   
1     0.062815     0.444209     0.062630     0.531393     0.060859   
2     0.062630     0.531393     0.060859     0.431934     0.063652   
3     0.060859     0.431934     0.063652     0.356585     0.060875   
4     0.063652     0.356585     0.060875     0.316960     0.063061   

      ...      var2(t+88)  var3(t+88)  var2(t+89)  var3(t+89)  var2(t+90)  \
0     ...        0.268147    0.011611    0.298976    0.011636    0.299144   
1     ...        0.298976    0.011636    0.299144    0.010164    0.254234   
2     ...        0.299144    0.010164    0.254234    0.009411    0.314873   
3     ...        0.254234    0.009411    0.314873    0.009418    0.375092   
4     ...        0.314873    0.009418    0.375092    0.006138    0.243912   

   var3(t+90)  Y1  Y2  Y3        Date  
0    0.010164   0   0   1  2008-08-06  
1    0.009411   0   0   1  2008-08-07  
2    0.009418   0   0   1  2008-08-08  
3    0.006138   0   0   1  2008-08-11  
4    0.006186   0   0   1  2008-08-12  

[5 rows x 486 columns]

# 3. Prepare dataframe

### Split into dependend and independent variables

In [40]:
X = df.drop(["Y1","Y2","Y3","Date"],axis=1).values
print(X.shape)

y = df.loc[:, df.columns.intersection(['Y1','Y2','Y3'])].values
print(y.shape)

(2368, 482)
(2368, 3)


In [41]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2)
print(X_train.shape,X_test.shape,y_train.shape,y_test.shape)

(1894, 482) (474, 482) (1894, 3) (474, 3)


# 4. Train and Test Neural Network using different hyperparameter

To train and test the neural network we will use GridsearchCV. 


**GridSearchCV:**
Using GridSeachCV we are able to fit, pedict and evaluate our model.
"The traditional way of performing hyperparameter optimization has been grid search [...] which is simply an exhaustive searching through a manually specified subset of the hyperparameter space of a learning algorithm." [Source](https://en.wikipedia.org/wiki/Hyperparameter_optimization)

To estimate this generalization performance we will use Cross Validation (CV).



To run the following code it is recommended to run it on a Cloud server e.g. on Amazon Web Service (AWS)

##### Define model 

In [17]:
# Define model
def create_model(optimizer='adam',init_mode1='uniform',
                 init_mode_output='uniform',activation_function1="linear",
                dropout_rate=0.0, weight_constraint=0, 
                loss='mean_squared_error',activation_output='linear'):
    
    model = Sequential()
                            
    #Setup neural network
    model.add(Dense(500, input_dim=482, activation=activation_function1, kernel_initializer=init_mode1, 
                    kernel_constraint=maxnorm(weight_constraint)))
    model.add(Dropout(dropout_rate))
    model.add(Dense(100, activation=activation_function1, kernel_initializer=init_mode1, 
                    kernel_constraint=maxnorm(weight_constraint)))
    model.add(Dropout(dropout_rate))
    model.add(Dense(50, activation=activation_function1, kernel_initializer=init_mode1, 
                    kernel_constraint=maxnorm(weight_constraint)))
    model.add(Dropout(dropout_rate))
    model.add(Dense(10, activation=activation_function1, kernel_initializer=init_mode1, 
                    kernel_constraint=maxnorm(weight_constraint)))
    model.add(Dropout(dropout_rate))
    model.add(Dense(3, activation=activation_output, kernel_initializer=init_mode_output))

    # Compile model
    model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['accuracy'])
    
    return model



##### Define parameter and run GridSeachCV

In [ ]:
# define the grid search parameters

# Batch Size
batch_size = [10, 100]

# Optimizer
optimizer = ['SGD', 'Adam']

# Network Weight Initialization
init_mode1 = ['uniform', 'normal', 'zero']
init_mode_output = ['uniform', 'normal', 'zero']

# Activation function
activation1 = ['relu', 'tanh', 'sigmoid', 'linear']
activation_output = ['relu', 'tanh', 'sigmoid', 'linear']

# Dropout Regularization
weight_constraint = [1, 2, 3]
dropout_rate = [0.0, 0.1, 0.2]



# create model
model = KerasClassifier(build_fn=create_model, epochs=50, verbose=2)

param_grid = dict(batch_size=batch_size, init_mode1=init_mode1, init_mode_output=init_mode_output, 
                  optimizer=optimizer,activation_function1=activation1,activation_output=activation_output,
                  dropout_rate=dropout_rate, weight_constraint=weight_constraint)

# CV k=3 (standard)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1, verbose=2)
grid_result = grid.fit(X, y)

Fitting 3 folds for each of 5184 candidates, totalling 15552 fits
[CV] activation_function1=relu, activation_output=relu, batch_size=10, dropout_rate=0.0, init_mode1=uniform, init_mode_output=uniform, optimizer=SGD, weight_constraint=1 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Epoch 1/50
 - 1s - loss: 0.2039 - acc: 0.8080
Epoch 2/50
 - 0s - loss: 0.1220 - acc: 0.8080
Epoch 3/50
 - 0s - loss: 0.1122 - acc: 0.8080
Epoch 4/50
 - 0s - loss: 0.1111 - acc: 0.8080
Epoch 5/50
 - 0s - loss: 0.1110 - acc: 0.8080
Epoch 6/50
 - 0s - loss: 0.1109 - acc: 0.8080
Epoch 7/50
 - 0s - loss: 0.1109 - acc: 0.8080
Epoch 8/50
 - 0s - loss: 0.1109 - acc: 0.8080
Epoch 9/50
 - 0s - loss: 0.1109 - acc: 0.8080
Epoch 10/50
 - 1s - loss: 0.1109 - acc: 0.8080
Epoch 11/50
 - 0s - loss: 0.1109 - acc: 0.8080
Epoch 12/50
 - 0s - loss: 0.1109 - acc: 0.8080
Epoch 13/50
 - 0s - loss: 0.1109 - acc: 0.8080
Epoch 14/50
 - 0s - loss: 0.1109 - acc: 0.8080
Epoch 15/50
 - 0s - loss: 0.1109 - acc: 0.8080
Epoch 16/50
 - 0s - loss: 0.1109 - acc: 0.8080
Epoch 17/50
 - 0s - loss: 0.1109 - acc: 0.8080
Epoch 18/50
 - 0s - loss: 0.1109 - acc: 0.8080
Epoch 19/50
 - 0s - loss: 0.1109 - acc: 0.8080
Epoch 20/50
 - 0s - loss: 0.1109 - acc: 0.8080
Epoch 21/50
 - 0s - loss: 0.1109 - acc: 0.8080
Epoch 22/50
 - 0s - lo

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   22.4s remaining:    0.0s


Epoch 1/50
 - 1s - loss: 0.2193 - acc: 0.7277
Epoch 2/50
 - 0s - loss: 0.1455 - acc: 0.7353
Epoch 3/50
 - 0s - loss: 0.1365 - acc: 0.7353
Epoch 4/50
 - 0s - loss: 0.1354 - acc: 0.7353
Epoch 5/50
 - 0s - loss: 0.1353 - acc: 0.7353
Epoch 6/50
 - 0s - loss: 0.1353 - acc: 0.7353
Epoch 7/50
 - 0s - loss: 0.1352 - acc: 0.7353
Epoch 8/50
 - 0s - loss: 0.1352 - acc: 0.7353
Epoch 9/50
 - 0s - loss: 0.1352 - acc: 0.7353
Epoch 10/50
 - 0s - loss: 0.1352 - acc: 0.7353
Epoch 11/50
 - 0s - loss: 0.1352 - acc: 0.7353
Epoch 12/50
 - 0s - loss: 0.1352 - acc: 0.7353
Epoch 13/50
 - 0s - loss: 0.1352 - acc: 0.7353
Epoch 14/50
 - 0s - loss: 0.1352 - acc: 0.7353
Epoch 15/50
 - 0s - loss: 0.1352 - acc: 0.7353
Epoch 16/50
 - 0s - loss: 0.1352 - acc: 0.7353
Epoch 17/50
 - 0s - loss: 0.1352 - acc: 0.7353
Epoch 18/50
 - 0s - loss: 0.1352 - acc: 0.7353
Epoch 19/50
 - 0s - loss: 0.1352 - acc: 0.7353
Epoch 20/50
 - 0s - loss: 0.1352 - acc: 0.7353
Epoch 21/50
 - 0s - loss: 0.1352 - acc: 0.7353
Epoch 22/50
 - 0s - lo

...

Epoch 45/50
 - 1s - loss: 0.3333 - acc: 0.2293
Epoch 46/50
 - 1s - loss: 0.3333 - acc: 0.2293
Epoch 47/50
 - 1s - loss: 0.3333 - acc: 0.2293
Epoch 48/50
 - 1s - loss: 0.3333 - acc: 0.2293
Epoch 49/50
 - 1s - loss: 0.3333 - acc: 0.2293
Epoch 50/50
 - 1s - loss: 0.3333 - acc: 0.2293
[CV]  activation_function1=relu, activation_output=relu, batch_size=100, dropout_rate=0.1, init_mode1=zero, init_mode_output=uniform, optimizer=SGD, weight_constraint=2, total= 2.5min
[CV] activation_function1=relu, activation_output=relu, batch_size=100, dropout_rate=0.1, init_mode1=zero, init_mode_output=uniform, optimizer=SGD, weight_constraint=2 
Epoch 1/50
 - 85s - loss: 0.3333 - acc: 0.2711
Epoch 2/50
 - 1s - loss: 0.3333 - acc: 0.2711
Epoch 3/50
 - 1s - loss: 0.3333 - acc: 0.2711
Epoch 4/50
 - 1s - loss: 0.3333 - acc: 0.2711
Epoch 5/50
 - 1s - loss: 0.3333 - acc: 0.2711
Epoch 6/50
 - 1s - loss: 0.3333 - acc: 0.2711
Epoch 7/50
 - 1s - loss: 0.3333 - acc: 0.2711
Epoch 8/50
 - 1s - loss: 0.3333 - acc: 0.2

Epoch 48/50
 - 1s - loss: 0.3333 - acc: 0.2293
Epoch 49/50
 - 1s - loss: 0.3333 - acc: 0.2293
Epoch 50/50
 - 1s - loss: 0.3333 - acc: 0.2293
[CV]  activation_function1=relu, activation_output=relu, batch_size=100, dropout_rate=0.1, init_mode1=zero, init_mode_output=uniform, optimizer=SGD, weight_constraint=3, total= 2.7min
[CV] activation_function1=relu, activation_output=relu, batch_size=100, dropout_rate=0.1, init_mode1=zero, init_mode_output=uniform, optimizer=SGD, weight_constraint=3 
Epoch 1/50
 - 91s - loss: 0.3333 - acc: 0.2711
Epoch 2/50
 - 1s - loss: 0.3333 - acc: 0.2711
Epoch 3/50
 - 1s - loss: 0.3333 - acc: 0.2711
Epoch 4/50
 - 1s - loss: 0.3333 - acc: 0.2711
Epoch 5/50
 - 1s - loss: 0.3333 - acc: 0.2711
Epoch 6/50
 - 1s - loss: 0.3333 - acc: 0.2711
Epoch 7/50
 - 1s - loss: 0.3333 - acc: 0.2711
Epoch 8/50
 - 1s - loss: 0.3333 - acc: 0.2711
Epoch 9/50
 - 1s - loss: 0.3333 - acc: 0.2711
Epoch 10/50
 - 1s - loss: 0.3333 - acc: 0.2711
Epoch 11/50
 - 1s - loss: 0.3333 - acc: 0.27

 - 1s - loss: 0.3333 - acc: 0.2293
[CV]  activation_function1=relu, activation_output=relu, batch_size=100, dropout_rate=0.1, init_mode1=zero, init_mode_output=uniform, optimizer=Adam, weight_constraint=1, total= 2.7min
[CV] activation_function1=relu, activation_output=relu, batch_size=100, dropout_rate=0.1, init_mode1=zero, init_mode_output=uniform, optimizer=Adam, weight_constraint=1 
Epoch 1/50
 - 98s - loss: 0.3333 - acc: 0.2711
Epoch 2/50
 - 1s - loss: 0.3333 - acc: 0.2711
Epoch 3/50
 - 1s - loss: 0.3333 - acc: 0.2711
Epoch 4/50
 - 1s - loss: 0.3333 - acc: 0.2711
Epoch 5/50
 - 1s - loss: 0.3333 - acc: 0.2711
Epoch 6/50
 - 1s - loss: 0.3333 - acc: 0.2711
Epoch 7/50
 - 1s - loss: 0.3333 - acc: 0.2711
Epoch 8/50
 - 1s - loss: 0.3333 - acc: 0.2711
Epoch 9/50
 - 1s - loss: 0.3333 - acc: 0.2711
Epoch 10/50
 - 1s - loss: 0.3333 - acc: 0.2711
Epoch 11/50
 - 1s - loss: 0.3333 - acc: 0.2711
Epoch 12/50
 - 1s - loss: 0.3333 - acc: 0.2711
Epoch 13/50
 - 1s - loss: 0.3333 - acc: 0.2711
Epoch 1

Epoch 1/50
 - 96s - loss: 0.3333 - acc: 0.2711
Epoch 2/50
 - 1s - loss: 0.3333 - acc: 0.2711
Epoch 3/50
 - 1s - loss: 0.3333 - acc: 0.2711
Epoch 4/50
 - 1s - loss: 0.3333 - acc: 0.2711
Epoch 5/50
 - 1s - loss: 0.3333 - acc: 0.2711
Epoch 6/50
 - 1s - loss: 0.3333 - acc: 0.2711
Epoch 7/50
 - 1s - loss: 0.3333 - acc: 0.2711
Epoch 8/50
 - 1s - loss: 0.3333 - acc: 0.2711
Epoch 9/50
 - 1s - loss: 0.3333 - acc: 0.2711
Epoch 10/50
 - 1s - loss: 0.3333 - acc: 0.2711
Epoch 11/50
 - 1s - loss: 0.3333 - acc: 0.2711
Epoch 12/50
 - 1s - loss: 0.3333 - acc: 0.2711
Epoch 13/50
 - 1s - loss: 0.3333 - acc: 0.2711
Epoch 14/50
 - 1s - loss: 0.3333 - acc: 0.2711
Epoch 15/50
 - 1s - loss: 0.3333 - acc: 0.2711
Epoch 16/50
 - 1s - loss: 0.3333 - acc: 0.2711
Epoch 17/50
 - 1s - loss: 0.3333 - acc: 0.2711
Epoch 18/50
 - 1s - loss: 0.3333 - acc: 0.2711
Epoch 19/50
 - 1s - loss: 0.3333 - acc: 0.2711
Epoch 20/50
 - 1s - loss: 0.3333 - acc: 0.2711
Epoch 21/50
 - 1s - loss: 0.3333 - acc: 0.2711
Epoch 22/50
 - 1s - l

 - 1s - loss: 0.3333 - acc: 0.2711
Epoch 4/50
 - 1s - loss: 0.3333 - acc: 0.2711
Epoch 5/50
 - 1s - loss: 0.3333 - acc: 0.2711
Epoch 6/50
 - 1s - loss: 0.3333 - acc: 0.2711
Epoch 7/50
 - 1s - loss: 0.3333 - acc: 0.2711
Epoch 8/50
 - 1s - loss: 0.3333 - acc: 0.2711
Epoch 9/50
 - 1s - loss: 0.3333 - acc: 0.2711
Epoch 10/50
 - 1s - loss: 0.3333 - acc: 0.2711
Epoch 11/50
 - 1s - loss: 0.3333 - acc: 0.2711
Epoch 12/50
 - 1s - loss: 0.3333 - acc: 0.2711
Epoch 13/50
 - 1s - loss: 0.3333 - acc: 0.2711
Epoch 14/50
 - 1s - loss: 0.3333 - acc: 0.2711
Epoch 15/50
 - 1s - loss: 0.3333 - acc: 0.2711
Epoch 16/50
 - 1s - loss: 0.3333 - acc: 0.2711
Epoch 17/50
 - 1s - loss: 0.3333 - acc: 0.2711
Epoch 18/50
 - 1s - loss: 0.3333 - acc: 0.2711
Epoch 19/50
 - 1s - loss: 0.3333 - acc: 0.2711
Epoch 20/50
 - 1s - loss: 0.3333 - acc: 0.2711
Epoch 21/50
 - 1s - loss: 0.3333 - acc: 0.2711
Epoch 22/50
 - 1s - loss: 0.3333 - acc: 0.2711
Epoch 23/50
 - 1s - loss: 0.3333 - acc: 0.2711
Epoch 24/50
 - 1s - loss: 0.333

##### Show the parameter which created the best result
The results are the results of the 

In [24]:
print(grid_result.best_params_)

{activation_function1=relu, activation_output=relu, batch_size=100, dropout_rate=0.1, init_mode1=normal, init_mode_output=normal, optimizer=Adam, weight_constraint=3}


### Fit "best" model

In [42]:
model = Sequential()
                            
#Setup neural network
model.add(Dense(500, input_dim=482, activation="relu", kernel_initializer="normal", 
                    kernel_constraint=maxnorm(3)))
model.add(Dropout(0.1))
model.add(Dense(100, activation="relu", kernel_initializer="normal", 
                    kernel_constraint=maxnorm(3)))
model.add(Dropout(0.1))
model.add(Dense(50, activation="relu", kernel_initializer="normal", 
                    kernel_constraint=maxnorm(3)))
model.add(Dropout(0.1))
model.add(Dense(10, activation="relu", kernel_initializer="normal", 
                    kernel_constraint=maxnorm(3)))
model.add(Dropout(0.1))
model.add(Dense(3, activation="relu", kernel_initializer="normal"))

    # Compile model
model.compile(loss='mean_squared_error', optimizer="Adam", metrics=['accuracy'])

neural_network = model.fit(X_train, y_train, epochs=100, batch_size=100, verbose=2, validation_data=(X_test, y_test))

Train on 1894 samples, validate on 474 samples
Epoch 1/100
 - 308s - loss: 0.2230 - acc: 0.6922 - val_loss: 0.1465 - val_acc: 0.7278
Epoch 2/100
 - 1s - loss: 0.1548 - acc: 0.7286 - val_loss: 0.1454 - val_acc: 0.7278
Epoch 3/100
 - 1s - loss: 0.1513 - acc: 0.7286 - val_loss: 0.1377 - val_acc: 0.7278
Epoch 4/100
 - 1s - loss: 0.1460 - acc: 0.7286 - val_loss: 0.1313 - val_acc: 0.7278
Epoch 5/100
 - 1s - loss: 0.1380 - acc: 0.7286 - val_loss: 0.1274 - val_acc: 0.7278
Epoch 6/100
 - 1s - loss: 0.1344 - acc: 0.7307 - val_loss: 0.1244 - val_acc: 0.7278
Epoch 7/100
 - 1s - loss: 0.1317 - acc: 0.7550 - val_loss: 0.1180 - val_acc: 0.7637
Epoch 8/100
 - 1s - loss: 0.1224 - acc: 0.7719 - val_loss: 0.1117 - val_acc: 0.7911
Epoch 9/100
 - 1s - loss: 0.1142 - acc: 0.7920 - val_loss: 0.1010 - val_acc: 0.8312
Epoch 10/100
 - 1s - loss: 0.1050 - acc: 0.8173 - val_loss: 0.1013 - val_acc: 0.8291
Epoch 11/100
 - 1s - loss: 0.1033 - acc: 0.8242 - val_loss: 0.0994 - val_acc: 0.8101
Epoch 12/100
 - 1s - loss

Epoch 97/100
 - 1s - loss: 0.0231 - acc: 0.9694 - val_loss: 0.0394 - val_acc: 0.9304
Epoch 98/100
 - 1s - loss: 0.0215 - acc: 0.9720 - val_loss: 0.0394 - val_acc: 0.9262
Epoch 99/100
 - 2s - loss: 0.0204 - acc: 0.9752 - val_loss: 0.0330 - val_acc: 0.9430
Epoch 100/100
 - 1s - loss: 0.0186 - acc: 0.9762 - val_loss: 0.0427 - val_acc: 0.9304


### Check the training and test accuracy

In [44]:
plt.plot(neural_network.history['acc'], label='train')
plt.plot(neural_network.history['val_acc'], label='test')
plt.legend()
plt.show()

# 5. Result

As you can see in the chart above we can receive pretty good results in our training and test set.

##### The best hyperparameter are:
activation_function1=relu, activation_output=relu, batch_size=100, dropout_rate=0.1, init_mode1=normal, init_mode_output=normal, optimizer=Adam, weight_constraint=3